# Train your first GAN model from scratch using PyTorch

## 参考资料
[1] [Train your first GAN model from scratch using PyTorch](https://blog.usejournal.com/train-your-first-gan-model-from-scratch-using-pytorch-9b72987fd2c0)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils import data as t_data
import torchvision.datasets as datasets
from torchvision import transforms

In [2]:
data_transforms = transforms.Compose([transforms.ToTensor()])
mnist_trainset = datasets.MNIST(root='./data', train=True,    
                           download=True, transform=data_transforms)
batch_size=4
dataloader_mnist_train = t_data.DataLoader(mnist_trainset, 
                                           batch_size=batch_size,
                                           shuffle=True
                                           )

0it [00:00, ?it/s]

100%|█████████▉| 9912320/9912422 [16:03<00:00, 5966.88it/s] 

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



0it [00:00, ?it/s]


 85%|████████▌ | 24576/28881 [00:05<00:00, 5894.60it/s]
32768it [00:06, 5408.42it/s]                           

0it [00:00, ?it/s]

...



  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|          | 16384/1648877 [00:01<00:51, 31723.22it/s]

  1%|▏         | 24576/1648877 [00:01<01:10, 23146.79it/s]

  2%|▏         | 40960/1648877 [00:01<00:53, 29908.77it/s]

  3%|▎         | 49152/1648877 [00:02<00:51, 30890.70it/s]

  4%|▍         | 65536/1648877 [00:02<00:43, 36760.50it/s]

  4%|▍         | 73728/1648877 [00:02<00:44, 35577.88it/s]

  5%|▍         | 81920/1648877 [00:02<00:54, 28737.21it/s]

  5%|▌         | 90112/1648877 [00:03<01:05, 23675.24it/s]

  6%|▋         | 106496/1648877 [00:03<00:59, 25808.69it/s]

  7%|▋         | 114688/1648877 [00:04<01:00, 25549.99it/s]

  7%|▋         | 122880/1648877 [00:04<01:16, 19843.56it/s]

  8%|▊         | 131072/1648877 [00:05<01:14, 20441.78it/s]

  8%|▊         | 139264/1648877 [00:05<01:09, 21655.54it/s]

  9%|▉         | 147456/1648877 [00:06<01:33, 16114.73it/s]

  9%|▉         | 155648/1648877 [00:06<01:32, 16186.64it/s]

 10%|▉         | 163840/1648877 [00:08<02:50, 

 63%|██████▎   | 1040384/1648877 [02:05<01:10, 8683.22it/s]

 64%|██████▎   | 1048576/1648877 [02:05<00:56, 10646.33it/s]

 64%|██████▍   | 1056768/1648877 [02:06<01:00, 9779.64it/s] 

 65%|██████▍   | 1064960/1648877 [02:06<00:52, 11154.72it/s]

 65%|██████▌   | 1073152/1648877 [02:07<00:56, 10133.80it/s]

 66%|██████▌   | 1081344/1648877 [02:08<00:46, 12322.99it/s]

 66%|██████▌   | 1089536/1648877 [02:09<00:57, 9797.48it/s] 

 67%|██████▋   | 1097728/1648877 [02:10<00:56, 9698.69it/s]

 67%|██████▋   | 1105920/1648877 [02:11<00:58, 9225.28it/s]

 68%|██████▊   | 1114112/1648877 [02:11<00:50, 10620.86it/s]

 68%|██████▊   | 1122304/1648877 [02:14<01:23, 6318.54it/s] 

 69%|██████▊   | 1130496/1648877 [02:14<01:06, 7754.60it/s]

 69%|██████▉   | 1138688/1648877 [02:15<00:55, 9209.56it/s]

 70%|██████▉   | 1146880/1648877 [02:15<00:47, 10604.32it/s]

 70%|███████   | 1155072/1648877 [02:17<00:54, 9012.03it/s] 

 71%|███████   | 1163264/1648877 [02:17<00:42, 11514.81it/s]

 71%|███████ 

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw





8192it [00:01, 7773.41it/s]             

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [3]:
def make_some_noise():
    return torch.rand(batch_size,100)

In [4]:
# defining generator class

class generator(nn.Module):
    
    def __init__(self, inp, out):
        
        super(generator, self).__init__()
        
        self.net = nn.Sequential(
                                 nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,1000),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(1000,800),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(800,out)
                                    )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [5]:
# defining discriminator class

class discriminator(nn.Module):
    
    def __init__(self, inp, out):
        
        super(discriminator, self).__init__()
        
        self.net = nn.Sequential(
                                 nn.Linear(inp,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,300),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(300,200),
                                 nn.ReLU(inplace=True),
                                 nn.Linear(200,out),
                                 nn.Sigmoid()
                                    )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [6]:
def plot_img(array,number=None):
    array = array.detach()
    array = array.reshape(28,28)
    
    plt.imshow(array,cmap='binary')
    plt.xticks([])
    plt.yticks([])
    if number:
        plt.xlabel(number,fontsize='x-large')
    plt.show()

In [8]:
d_steps = 100
g_steps = 100

# criteriond1 = nn.BCELoss()
# optimizerd1 = optim.SGD(dis.parameters(), lr=0.001, momentum=0.9)

criteriond2 = nn.BCELoss()
optimizerd2 = optim.SGD(gen.parameters(), lr=0.001, momentum=0.9)

printing_steps = 200

epochs = 50

NameError: name 'gen' is not defined

In [ ]:
for epoch in range(epochs):
    
    print epoch

    # training discriminator
    for d_step in range(d_steps):
        dis.zero_grad()
        
        # training discriminator on real data
        for inp_real,_ in dataloader_mnist_train:
            inp_real_x = inp_real
            break

        inp_real_x = inp_real_x.reshape(batch_size,784)
        dis_real_out = dis(inp_real_x)
        dis_real_loss = criteriond1(dis_real_out,
                              Variable(torch.ones(batch_size,1)))
        dis_real_loss.backward()

        # training discriminator on data produced by generator
        inp_fake_x_gen = make_some_noise()
        #output from generator is generated        
        dis_inp_fake_x = gen(inp_fake_x_gen).detach()
        dis_fake_out = dis(dis_inp_fake_x)
        dis_fake_loss = criteriond1(dis_fake_out,
                                Variable(torch.zeros(batch_size,1)))
        dis_fake_loss.backward()

        optimizerd1.step()
        
        
            
    # training generator
    for g_step in range(g_steps):
        gen.zero_grad()
        
        #generating data for input for generator
        gen_inp = make_some_noise()
        
        gen_out = gen(gen_inp)
        dis_out_gen_training = dis(gen_out)
        gen_loss = criteriond2(dis_out_gen_training,
                               Variable(torch.ones(batch_size,1)))
        gen_loss.backward()
        
        optimizerd2.step()
        
    if epoch%printing_steps==0:
        plot_img(gen_out[0])
        plot_img(gen_out[1])
        plot_img(gen_out[2])
        plot_img(gen_out[3])
        print("\n\n")

In [ ]:
plot_img(gen(make_some_noise()))



1654784it [03:19, 13988.53it/s]                             